In [ ]:
!pip install qiskit
!pip install qiskit_aer

In [ ]:
# Task 1 — Custom Oracle Design

!pip install qiskit qiskit-aer -q

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def deutsch_oracle_xxor1(qc):
    # Implements |x>|y> → |x>|y XOR (x XOR 1)>
    qc.x(1)      # Adds the +1 term
    qc.cx(0, 1)  # Adds XOR with x

# Build circuit
qc = QuantumCircuit(2, 1)
qc.x(1)                 # prepare output |1>
qc.h([0,1])             # Hadamard on both
qc.barrier()
deutsch_oracle_xxor1(qc)
qc.barrier()
qc.h(0)
qc.measure(0, 0)

print(qc.draw(output="text"))

# Simulate
sim = AerSimulator()
result = sim.run(transpile(qc, sim)).result()
counts = result.get_counts()
print("\nMeasurement:", counts)
print("Interpretation: 0 → constant, 1 → balanced")


In [ ]:
# Task 2 — Bloch Sphere Visualization
%matplotlib inline

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt

# After Hadamard
qc_h = QuantumCircuit(2)
qc_h.x(1)
qc_h.h([0,1])
sv_h = Statevector.from_instruction(qc_h)
print("Statevector after Hadamard:\n", sv_h)
plot_bloch_multivector(sv_h)
plt.show()

# After Oracle f(x)=x XOR 1
qc_o = QuantumCircuit(2)
qc_o.x(1)
qc_o.h([0,1])
qc_o.x(1)
qc_o.cx(0,1)
sv_o = Statevector.from_instruction(qc_o)
print("Statevector after Oracle:\n", sv_o)
plot_bloch_multivector(sv_o)
plt.show()


In [ ]:
#Task 3 — Deutsch–Jozsa Extension (2-Qubit Input)

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def oracle_constant_0(qc, n):
    pass  # f(x) = 0

def oracle_balanced_xor(qc, n):
    # f(x0,x1)=x0 XOR x1
    qc.cx(0, n)
    qc.cx(1, n)

def deutsch_jozsa(n, oracle_fn):
    qc = QuantumCircuit(n+1, n)
    qc.x(n)
    qc.h(range(n+1))
    qc.barrier()
    oracle_fn(qc, n)
    qc.barrier()
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

n = 2
sim = AerSimulator()

# Balanced oracle
qc_balanced = deutsch_jozsa(n, oracle_balanced_xor)
res_bal = sim.run(transpile(qc_balanced, sim)).result().get_counts()
print("Balanced oracle counts:", res_bal)

# Constant oracle
qc_const = deutsch_jozsa(n, oracle_constant_0)
res_const = sim.run(transpile(qc_const, sim)).result().get_counts()
print("Constant oracle counts:", res_const)


In [ ]:
# Task 4 — Noise Impact Analysis
!pip install qiskit qiskit-aer -q

# 🧪 Add quantum noise and analyze its impact
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Build a simple noise model
noise_model = NoiseModel()
p1, p2 = 0.001, 0.01  # noise probabilities
noise_model.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['x','h','sx','u1','u2','u3'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])

# Reuse previous qc_balanced (from Task 3)
sim_noisy = AerSimulator(noise_model=noise_model)
job = sim_noisy.run(transpile(qc_balanced, sim_noisy), shots=4096)
counts_noisy = job.result().get_counts()

print("Noisy measurement results:\n", counts_noisy)


In [ ]:
# Task 5 — Compare Classical vs Quantum Queries

def f_1bit_xxor1(x): return x ^ 1

def classical_det_check_1bit(eval_fn):
    qcount = 0
    r0 = eval_fn(0); qcount += 1
    r1 = eval_fn(1); qcount += 1
    result = "constant" if r0 == r1 else "balanced"
    return result, qcount

res_classical, queries = classical_det_check_1bit(f_1bit_xxor1)
print("Classical deterministic:", res_classical, "| queries used:", queries)
print("Quantum Deutsch uses only 1 query.\n")

def f_2bit_parity(x):
    b0 = x & 1
    b1 = (x >> 1) & 1
    return b0 ^ b1

def classical_worst_case_queries(n):
    return (2**(n-1)) + 1

print("Classical worst-case queries for n=2:", classical_worst_case_queries(2))
print("Quantum Deutsch–Jozsa queries:", 1)


In [ ]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [ ]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [ ]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [ ]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [ ]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)
